In [1]:
# import stuff
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.cross_validation import train_test_split, cross_val_predict
from scipy import stats
from sklearn.cross_validation import ShuffleSplit
import numpy as np
from collections import defaultdict
import seaborn as sns
import almlab_scripts as alm
import time
import os
import cPickle as pickle
from sklearn import linear_model
import warnings

%matplotlib inline

<h2> Get OTU table and metadata </h2>

In [19]:
# Get all the metadata and OTU tables and remove maturation project from them
reload(alm)
#set your local paths
my_path = '/home/irockafe/Dropbox (MIT)/Alm_Lab/Chicken_Project/'
output_base = my_path + '4Isaac/Lasso/'
# Get Metadata, rdp info, and otu table

# Get Metadata as dataframe
metadata_path = my_path + '/4Isaac/Metadata_files/allTogether_meta.txt'
metadata = pd.read_table(metadata_path, sep='\t', index_col=0, header=0)

# Load in the RDP information
rdp_path = my_path + '/4Isaac/Metadata_files/biom_attempts/CobBreeder_AllTogether_250.otu_seqs.97.rdp'
asdf = alm.get_phylo_from_RDP(rdp_path, 'denovo38', 0.5) # gets phylo data
print asdf

# Import the OTU table as a dataframe and normalize
otu_table_path = my_path + '/4Isaac/CobbBreeder_usearchQfilt_062016.otu_table.97.denovo.f500.txt'
otu_table = pd.read_table(otu_table_path, sep='\t', index_col=0, header=0)
# Make fractions
counts = otu_table.sum(axis=0)
fractional_otu_table = otu_table.div(counts, axis=1)
assert((1-fractional_otu_table.sum(axis=0) <= 1e-9)).all()
# log transform
max_reads = max(otu_table.sum(axis=0))
zero_filler = float(1/(max_reads*10))
log_otu_table = alm.log_transform(fractional_otu_table, zero_filler)

# Remove columns from metadata that aren't in OTU table
all_meta = metadata.loc[otu_table.columns] 


Phylum: Bacteroidetes


<h2> Divide OTU table into each experiment </h2>

In [25]:
# Subdivide the counts OTU tables by each project and store them in a dictionary so that we 
# can filter things correctly downstream (Example: We want to keep only OTUs 
# with >1e-3 abundance in at least 7 samples for Line A, Study#1. We don't want to do this
# for All samples, just those in Line A, Study#1)

lines = ['A', 'B']
features = ['WeightGr', 'FCR', 'FERES']
otu_tables = {}

for line in lines:
    for feature in features:
        series_name = 'Line_%s_%s' % (line, feature)
        # we don't care about some groupings
        if series_name in ['Line_B_FCR', 'Line_A_FCR', 'Line_A_FERES', 'Line_B_FERES']:
            continue
        group = all_meta[ (all_meta['Line'] == line) & 
                         (pd.notnull(all_meta[feature]))]
        metadata = group[feature]
        group_otus = otu_table[group.index]
        
        otu_tables[series_name] = {'otu table': group_otus, 'metadata': metadata}

# Repeat for FCR, FERES, Lines A&B combined
features = ['FCR', 'FERES']
for feature in features:
    group = all_meta[(pd.notnull(all_meta[feature]))]
    metadata = group[feature]
    group_otus = otu_table[group.index]
    series_name = '%s' % (feature)
    otu_tables[series_name] = {'otu table': group_otus, 'metadata': metadata} #assign to a dictionary, so I can access the correct table later

for i in otu_tables.keys():
    print '\n------%s-------' % i
    # Make sure you didn't screw up
    assert( sorted(otu_tables[i]['otu table'].columns) == sorted(otu_tables[i]['metadata'].index))


------Line_B_WeightGr-------

------Line_A_WeightGr-------

------FCR-------

------FERES-------


<h2> Now add the filtered OTU table and +/- lasso coefficients to each experimental group </h2>

In [30]:
# require OTU present in 4+ chickens to remain
# And load up coefficient stability-selection from Lasso cross-validation

filtered_otu_tables = {key:{'metadata': None, 
                            'otu table': None,
                           'pos coef': None} for key in otu_tables}

for key, value in otu_tables.iteritems():
    original_table = value['otu table']
    metadata = value['metadata']
    # filter by 4 chicken prevalence
    filtered_table = alm.filter_abundance(original_table, 0, 4)
    # add filtered table to your data
    otu_tables[key]['filtered table'] = filtered_table
    # add positive and negative cv-lasso coefficients to data
    # Get the coefficient data
    neg_coef_path = output_base + 'leave_one_out_cross_validation/coefficient_stability/%s_negative_coefficient_stability.tab' % key
    pos_coef_path = output_base + 'leave_one_out_cross_validation/coefficient_stability/%s_positive_coefficient_stability.tab' % key

    neg_coef_data = pd.read_csv(neg_coef_path, sep='\t', index_col=0)
    pos_coef_data = pd.read_csv(pos_coef_path, sep='\t', index_col=0)
    otu_tables[key]['+ lasso coef'] = pos_coef_data
    otu_tables[key]['- lasso coef'] = neg_coef_data


In [38]:
# Make null distribution for spearman, all samples and 4-chicken filtered
print otu_tables.keys()


['Line_B_WeightGr', 'Line_A_WeightGr', 'FCR', 'FERES']


In [ ]:
# *** Load up the coefficients selected by lasso

#*** Filter OTUs at 4-chicken prevalence level

# Multihypothesis test 4-chicken filtered OTU tables for 
# spearman, pearson, and mann-whitney

# Make boxplots with lasso-OTUs & empirical Mann-whitney value

# Make log-abundance vs. y plots with empirical spearman and pearson values
